In [30]:
import librosa
import librosa.display
import numpy as np
import soundfile as sf
from tqdm import tqdm 
import os
import numpy as np
import csv
from sklearn.mixture import GaussianMixture

link to the dataset 
https://drive.google.com/file/d/1G-6A60OhBvURa0GC3ry9FpvKI1mk1t2W/view

In [4]:
path="Reconnaissance_du_locuteur"

In [7]:
def remove_silence(audio_path, output_path, top_db=30):
    y, sr = librosa.load(audio_path, sr=None)
    non_silent_intervals = librosa.effects.split(y, top_db=top_db)
    y_trimmed = np.concatenate([y[start:end] for start, end in non_silent_intervals])
    sf.write(output_path, y_trimmed, sr)

In [9]:
for speaker in tqdm(os.listdir(path), desc="Processing Speakers"):
    speaker_path = os.path.join(path, speaker)
    for subfolder in ["train", "test"]:
        subfolder_path = os.path.join(speaker_path, subfolder)
        if subfolder == "test":
                    # Process the 5s, 10s, 15s subfolders
                    for duration in ["5s", "10s", "15s"]:
                        duration_path = os.path.join(subfolder_path, duration)
                        for file in os.listdir(duration_path):
                                input_path = os.path.join(duration_path, file)
                                output_path = os.path.join(duration_path, f"cleaned_{file}")
                                remove_silence(input_path, output_path)
        else:
            for file in os.listdir(subfolder_path):
                            input_path = os.path.join(subfolder_path, file)
                            output_path = os.path.join(subfolder_path, f"cleaned_{file}")
                            remove_silence(input_path, output_path)

Processing Speakers: 100%|█████████████████████████████████████████████████████████████| 23/23 [01:04<00:00,  2.81s/it]


In [15]:
'''
def extract_mfcc(audio_path, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=None)
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return np.mean(mfccs, axis=1)  # Taking mean across time axis

def process_cleaned_train(root_path, output_csv):
    data = []
    
    for speaker_folder in tqdm(os.listdir(root_path), desc="Processing speakers"):
        speaker_path = os.path.join(root_path, speaker_folder)
        train_path = os.path.join(speaker_path, "Train")
        
        if os.path.exists(train_path):  # Ensure 'Train' folder exists
            cleaned_audio_path = os.path.join(train_path, "cleaned_Train.wav")  # Adjust if extension varies
            
            if os.path.exists(cleaned_audio_path):  # Ensure the file exists
                mfcc_features = extract_mfcc(cleaned_audio_path)
                data.append([speaker_folder] + mfcc_features.tolist())

    columns = ["Speaker"] + [f"MFCC_{i+1}" for i in range(len(mfcc_features))]
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"MFCC features saved to {output_csv}")

root_path = path
output_csv = "mfcc_features.csv"
process_cleaned_train(root_path, output_csv)
'''

Processing speakers: 100%|█████████████████████████████████████████████████████████████| 23/23 [00:11<00:00,  1.93it/s]

MFCC features saved to mfcc_features.csv


In [45]:
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

def extract_and_process_mfcc(audio_path, n_mfcc=13):
    """Extract MFCCs, remove low-energy frames, and return the mean across time axis."""
    # Load audio
    y, sr = librosa.load(audio_path, sr=None)
    
    # Extract MFCCs
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    
    # Remove the first coefficient (energy)
    mfcc = mfcc[1:, :]
    
    # Return the mean of MFCCs across time axis
    return np.mean(mfcc, axis=1)

def process_cleaned_train(root_path, output_csv, n_mfcc=13):
    data = []
    
    for speaker_folder in tqdm(os.listdir(root_path), desc="Processing speakers"):
        speaker_path = os.path.join(root_path, speaker_folder)
        train_path = os.path.join(speaker_path, "Train")
        
        if os.path.exists(train_path):  # Ensure 'Train' folder exists
            cleaned_audio_path = os.path.join(train_path, "cleaned_Train.wav")  # Adjust if extension varies
            
            if os.path.exists(cleaned_audio_path):  # Ensure the file exists
                # Extract MFCC features and compute the mean
                mfcc_features = extract_and_process_mfcc(cleaned_audio_path, n_mfcc)
                data.append([speaker_folder] + mfcc_features.tolist())
    
    # Create DataFrame and save to CSV
    columns = ["Speaker"] + [f"MFCC_{i+1}" for i in range(n_mfcc-1)]  # n_mfcc-1 because the first coefficient is dropped
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"MFCC features saved to {output_csv}")

# Path setup
root_path = path  # Replace with the actual root path
output_csv = "mfcc_features2.csv"
process_cleaned_train(root_path, output_csv)


Processing speakers: 100%|█████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.82it/s]

MFCC features saved to mfcc_features2.csv


In [54]:
import librosa
import numpy as np
import os
import pandas as pd
from tqdm import tqdm

def extract_mfcc(audio_path, n_mfcc=13):
    y, sr = librosa.load(audio_path, sr=None)
    # Extract MFCC features frame-by-frame
    mfccs = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=n_mfcc)
    return mfccs.T  # Transpose to have shape (n_frames, n_mfcc)

def process_cleaned_train(root_path, output_csv, n_mfcc=13):
    data = []
    
    for speaker_folder in tqdm(os.listdir(root_path), desc="Processing speakers"):
        speaker_path = os.path.join(root_path, speaker_folder)
        train_path = os.path.join(speaker_path, "Train")
        
        if os.path.exists(train_path):  # Ensure 'Train' folder exists
            cleaned_audio_path = os.path.join(train_path, "cleaned_Train.wav")  # Adjust if extension varies
            
            if os.path.exists(cleaned_audio_path):  # Ensure the file exists
                # Extract MFCC features from all frames
                mfcc_frames = extract_mfcc(cleaned_audio_path, n_mfcc)
                
                # Add each MFCC frame to the data list
                for frame in mfcc_frames:
                    data.append([speaker_folder] + frame.tolist())  # Append each frame for the speaker

    # Generate columns for the DataFrame
    columns = ["Speaker"] + [f"MFCC_{i+1}" for i in range(n_mfcc)]
    
    # Create DataFrame and save to CSV
    df = pd.DataFrame(data, columns=columns)
    df.to_csv(output_csv, index=False)
    print(f"MFCC features saved to {output_csv}")

# Path setup
root_path = path  # Replace with the actual root path
output_csv = "mfcc_features2.csv"
process_cleaned_train(root_path, output_csv)


Processing speakers: 100%|█████████████████████████████████████████████████████████████| 23/23 [00:06<00:00,  3.56it/s]


MFCC features saved to mfcc_features2.csv


In [24]:
mfcc_csv = "mfcc_features2.csv"  # Remplace par le chemin de ton fichier CSV
df = pd.read_csv(mfcc_csv)

In [49]:
gmm_root = "gmm"
os.makedirs(gmm_root, exist_ok=True)
gaussiennes_list = [8, 16, 32, 64, 128]

In [51]:
for speaker in tqdm(df["Speaker"].unique(), desc="Training"):
    speaker_data = df[df["Speaker"] == speaker].iloc[:, 1:].values  # Exclure la colonne 'Speaker'
    
    if speaker_data.shape[0] == 0:
        print(f"Aucun MFCC trouvé pour {speaker}, skipping...")
        continue

    speaker_folder = os.path.join(gmm_root, speaker)
    os.makedirs(speaker_folder, exist_ok=True)

    for gaussiennes in gaussiennes_list:
        gmm = GaussianMixture(n_components=gaussiennes, covariance_type='diag', n_init=3, random_state=42)
        gmm.fit(speaker_data)  # Entraîner le modèle GMM

        # Sauvegarde du modèle
        model_filename = os.path.join(speaker_folder, f"GMM_{n_components}.pkl")
        with open(model_filename, "wb") as model_file:
            pickle.dump(gmm, model_file)

print(f"Tous les modèles GMM sont sauvegardés dans {gmm_root}/")

Training:   0%|                                                                                 | 0/23 [00:00<?, ?it/s]


ValueError: Found array with 1 sample(s) (shape=(1, 13)) while a minimum of 2 is required by GaussianMixture.

In [37]:
import pandas as pd

# Charger les MFCCs
mfcc_csv = "mfcc_features.csv"  # Mets le bon chemin
df = pd.read_csv(mfcc_csv)

# Vérifier le nombre d'échantillons par locuteur
speaker_counts = df["Speaker"].value_counts()

# Afficher les locuteurs qui ont moins de 2 échantillons
print(speaker_counts[speaker_counts < 2])


Speaker
F1     1
F9     1
H7     1
H6     1
H5     1
H4     1
H3     1
H2     1
H11    1
H10    1
H1     1
F8     1
F10    1
F7     1
F6     1
F5     1
F4     1
F3     1
F2     1
F13    1
F12    1
F11    1
H9     1
Name: count, dtype: int64
